In [149]:
%load_ext autoreload
%autoreload 2

import torch
from torch.utils.data import DataLoader
from models.transformer import Transformer
from dataloader import EnglishSpanish
from train import Trainer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Model

In [150]:
data = EnglishSpanish()

Corpus src: 12
Corpus tgt: 13


In [151]:
tm = Transformer(data.voc_src_len, data.voc_tgt_len)

In [152]:
trainer = Trainer(iters=20)
_, _ = trainer.fit(tm, data)

0/1 | loss: 0.9108041524887085
EPOCH 1 train loss: 0.9108041524887085

0/1 | loss: 0.7191761136054993
EPOCH 2 train loss: 0.7191761136054993

0/1 | loss: 0.6274900436401367
EPOCH 3 train loss: 0.6274900436401367

0/1 | loss: 0.5647739768028259
EPOCH 4 train loss: 0.5647739768028259

0/1 | loss: 0.509441077709198
EPOCH 5 train loss: 0.509441077709198

0/1 | loss: 0.4431358873844147
EPOCH 6 train loss: 0.4431358873844147

0/1 | loss: 0.38381922245025635
EPOCH 7 train loss: 0.38381922245025635

0/1 | loss: 0.33484014868736267
EPOCH 8 train loss: 0.33484014868736267

0/1 | loss: 0.2945493459701538
EPOCH 9 train loss: 0.2945493459701538

0/1 | loss: 0.26122957468032837
EPOCH 10 train loss: 0.26122957468032837

0/1 | loss: 0.23324202001094818
EPOCH 11 train loss: 0.23324202001094818

0/1 | loss: 0.21175462007522583
EPOCH 12 train loss: 0.21175462007522583

0/1 | loss: 0.19159573316574097
EPOCH 13 train loss: 0.19159573316574097

0/1 | loss: 0.17370697855949402
EPOCH 14 train loss: 0.17370697

# Inference

## Custom sentence

In [153]:
sentence = "ey! hola"
sentence = data._tokenize_es(sentence)
inp_pad = data.MAX_LEN - len(sentence)
sentence = sentence + ["<PAD>"] * inp_pad
inp = data._token_to_idx(sentence).reshape(1, -1)

## Sentence from training

In [154]:
dl = iter(DataLoader(data))

In [155]:
inp, out, inp_pad, out_pad, src, tgt = next(dl)
print(data._idx_to_token(inp, is_src=True))
print(data._idx_to_token(out))

[['hola', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']]
[['hi', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']]


## Inference

In [156]:
tm.eval()
sos = torch.repeat_interleave(data.sos, 1, 0)
out1 = torch.cat([sos], 1)
print(out1)
print(data._idx_to_token(out1))

tensor([[0]])
[['<SOS>']]


In [158]:
PICK_TOP = 0

R = tm(inp, out1, inp_pad, [0])
last_pred = R[0,-1]
values, indices = torch.topk(last_pred, k=5)
probs = values.detach().numpy()
r = torch.tensor([indices[PICK_TOP]])
out1 = torch.cat([out1, r.reshape(1, 1)], 1)
print("Top best values:")
print([(data.voc_tgt[x], p) for x, p in zip(indices, probs)])
print()
print("Current output:")
print(data._idx_to_token(out1))

Top best values:
[('<EOS>', 0.9490163), ('hi', 0.0110339485), ('!', 0.008259185), (',', 0.008106744), ('<PAD>', 0.004780334)]

Current output:
[['<SOS>', 'hi', '<EOS>']]
